# Extract the quadrat from all still images in dataset

Prior to running this notebook you must set the `DATA_PATH` environment variable 

In [ ]:
import os
import os.path as osp

import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage import transform
from glob import glob

# local import
from utils import *

In [ ]:
# you must set 
DATA_PATH = osp.join(os.environ['DATA_PATH'], 'cciw/Data')

In [ ]:
# Search for all still files in the datset
stills = glob(os.path.join(DATA_PATH,'Videos_and_stills/GLNI/*/*/*/Stills/Quad*/*.jpg'))
len(stills)

In [ ]:
i = 34
#for i in range(i, len(stills)):
for i in range(i, i + 1):
    bgr = cv2.imread(stills[i])
    print('Got ', bgr.shape)

    # set parameters according to input resolution
    if bgr.shape[1] == 1440:

        """x_trim and y_trim are used to remove black padding
        for 1080x1440 video which triggers spurious edges"""
        x_trim, y_trim = 1, 145
        bgr = bgr[y_trim:-y_trim, x_trim:-x_trim, :]
        crop_frame_border = True

        """@param canny_thresh# hysteresis values for Canny edge 
        detector, input to HoughLines"""
        canny_thresh1, canny_thresh2 = 10, 45
        """@param threshold Accumulator threshold, return 
        lines with more than threshold of votes. (intersection points)"""
        threshold = 125
        """@param minLineLength Minimum line length. 
        Line segments shorter than that are rejected. (pixels)"""
        mLL = 400
        """@param maxLineGap Maximum allowed gap between points 
        on the same line to link them. (pixels)"""
        mLG = 150

    elif bgr.shape[1] == 1920:
        """@param canny_thresh# hysteresis values for Canny edge 
        detector, input to HoughLines"""
        #canny_thresh1, canny_thresh2 = 40, 600
        #canny_thresh1, canny_thresh2 = 40, 300
        canny_thresh1, canny_thresh2 = 40, 100
        """@param threshold Accumulator threshold, return 
        lines with more than threshold of votes. (intersection points)"""
        threshold = 125
        """@param minLineLength Minimum line length. 
        Line segments shorter than that are rejected. (pixels)"""
        mLL = 500
        """@param maxLineGap Maximum allowed gap between points 
        on the same line to link them. (pixels)"""
        mLG = 150    

        for j in range(bgr.shape[1]):
            if bgr[:, j].mean() > 1:
                break
        h_pad = 10
        bgr = bgr[:, j + h_pad:-(h_pad + j), :]

    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

    bgr, edges, corners = draw_all_lines(
        bgr, mll=mLL, mlg=mLG, threshold=threshold, canny_1=canny_thresh1, canny_2=canny_thresh2)

    #fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 10))
    fig, ax1 = plt.subplots(1, 1, figsize=(14, 10))
    ax1.imshow(bgr)
    #ax2.imshow(edges)
    ax1.set_title(i)#'%d, canny_2 %.1f' % (i, c2))
    plt.show()

In [ ]:
centroid = corners.mean(axis=0, keepdims=True)[0]
corner_dist = np.linalg.norm(corners - centroid, axis=1)
print(corner_dist)
indices = np.argsort(corner_dist)
crop = corners[indices][:4].astype('int')

In [ ]:
def draw_all_lines(bgr, rho=1, theta=np.pi/90, mll=300, mlg=100, threshold=100, ds=1, canny_1=30, canny_2=400, outlier_angle_thresh=20):
    """Draw Hough lines and corner points on image 'im'
    
    @param rho -- Distance resolution of the accumulator (pixels).
    @param theta -- Angle resolution of the accumulator (radians).
    @param threshold -- Accumulator threshold, return lines with 
                        more than threshold of votes. (intersection points)
    @param minLineLength -- Minimum line length. Line segments shorter than 
                            that are rejected. (pixels)
    @param maxLineGap -- Maximum allowed gap between points on the same line 
                         to link them. (pixels)
    @param canny_threshold1 Histeresis threshold 1
    @param canny_threshold2
    """
    trim_px = 3
    DP1, DP2 = 0, 1
    X, Y = 0, 1
    
    img = np.ascontiguousarray(bgr[::ds, ::ds], dtype=np.uint8)
    img_h = img.shape[0]
    img_w = img.shape[1]
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    '''
    scale_percent = 75 # percent of original size
    width = int(img_w * scale_percent / 100)
    height = int(img_h * scale_percent / 100)
    dim = (width, height)
    # resize image
    img = cv2.resize(img, dim)    
    '''
    
    linesP = [i for i in np.arange(17)]

    while len(linesP) > 14:
        # run the Canny edge detector on the rotated gray scale image
        edges = cv2.Canny(img, threshold1=canny_1, threshold2=canny_2, L2gradient=False)
        edges = edges[trim_px:-trim_px, trim_px:-trim_px]
        # run the probabilistic hough lines transform
        linesP = cv2.HoughLinesP(edges, rho, theta, threshold=threshold, minLineLength=mll, maxLineGap=mlg)
        if linesP is None:
            break
        canny_2 += 10
    
    img = img[trim_px:-trim_px, trim_px:-trim_px, :]
    img = np.ascontiguousarray(img, dtype=np.uint8)
    img_h = img.shape[0]
    img_w = img.shape[1]

    # @param lines The extremes of the detected lines if any (<N_LINES_FOUND>, 1, x_0, y_0, x_1, y_1). (pixels)\
    if linesP is not None:
        print('Found %d lines' % len(linesP))    
        coords = np.zeros((len(linesP), 2, 2)).astype('int') # points, start/end, x/y
        for i in range(len(linesP)):
            l = linesP[i][0]
            coords[i, DP1, X] = l[0] # x1
            coords[i, DP1, Y] = l[1] # y1
            coords[i, DP2, X] = l[2] # x2
            coords[i, DP2, Y] = l[3] # y2

        # clean up coordinates
        if len(linesP) > 1:
            coords = reject_outlier_lines(coords, outlier_angle_thresh)
            coords = merge_noisy_lines(coords.astype('int64'))

        # find all intersection points
        if len(linesP) > 1:
            corners_np = get_intersection_pts(coords, img_w, img_h)
            for i in range(len(corners_np)):
                cv2.circle(img, (int(corners_np[i, 0]), 
                                 int(corners_np[i, 1]) ), 10,
                           (0, 0, 255), thickness=2, lineType=8, shift=0)
        
        # attempt to rotate the image
        if len(coords) > 4:
            angles = []
            for j in range(len(c)):
                dx = c[j][DP2][X] - c[j][DP1][X]
                dy = c[j][DP2][Y] - c[j][DP1][Y]
                a = np.round(np.rad2deg(np.arctan2(dy, dx)))
                a = a - 90 if a > 90 else a
                angles.append(a)
            angles = np.asarray(angles)    
            rot_deg = -angles.min()
            
            # create a rotation matrix to rotate the coord matrix
            t = np.deg2rad(rot_deg)
            center = (coords[0, DP1, X], coords[0, DP1, Y])
            
            R = np.array([[np.cos(t), -np.sin(t)],
                          [np.sin(t),  np.cos(t)]])
            coordsR = np.dot(coords, R)
            correction_factor = coords[0, DP1, Y] - coordsR[0, DP1, Y]
            for i in range(len(coordsR)):
                coordsR[i, DP1, Y] += correction_factor
                coordsR[i, DP2, Y] += correction_factor
            
            img = transform.rotate(
                np.ascontiguousarray(img, dtype=np.uint8), rot_deg, center=center, resize=False)
            img = (255 * img).astype(np.uint8)
            
            # draw lines on canvas from coordsR
            img = draw_lines_from_coords(img, coordsR)
        else:
            # draw lines on canvas
            img = draw_lines_from_coords(img, coords)

        '''
        if len(corners_np) > 3:
            # plot centroid
            centroid, crop = centroid_and_crop_pts(corners_np)
            cv2.circle(img, (int(centroid[0]), int(centroid[1])), 20, 
                           (0, 0, 0), thickness=6, lineType=8, shift=0)
            for i in range(len(crop)):
                cv2.circle(img, ( int(crop[i, 0]), int(crop[i, 1]) ), 10, 
                           (255, 255, 255), thickness=2, lineType=8, shift=0)
        '''
    return img, edges, corners_np

In [ ]:
#for i in range(len(stills)):
i = 29
bgr = cv2.imread(stills[i])
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10, 8))
plt.imshow(bgr[::2, ::2, :])
plt.title(i)
plt.show()